<a href="https://colab.research.google.com/github/daphnecor/NeuroAnalysis/blob/main/NA4_CalciumImaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neuro Analysis 4: Calcium imaging

```
Student: Daphne Cornelisse 1066862
```

In [ ]:
##import python standard libraries
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle
import psutil
import logging
import os

##import caiman and caiman funcions that will be used
import caiman as cm
from caiman.motion_correction import MotionCorrect, high_pass_filter_space
from caiman.source_extraction.cnmf import params as params
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.base.movies import load_movie_chain

##import auxiliary functions for plotting file
import auxiliary_functions
from ipywidgets import interact
import ipywidgets as widgets

from tqdm import tqdm as tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import clear_output

In [ ]:
data_dir = '/Users/Daphne/Desktop/data/'           # define your data directory
figures_dir = '/Users/Daphne/Desktop/figures/'     # define your results directory

In [ ]:
input_tif_file_path_FOV = data_dir + 'raw/' + 'calcium_video.tif'   # define the path of the raw FOV movie
if not os.path.isfile(input_tif_file_path_FOV):                     # verify that the file exist
    logging.error('Calcium video file not found.')

input_tif_file_path = data_dir + 'raw/' + 'caiman_video_trial_0.tif'   # define the path of the raw ROI movie
if not os.path.isfile(input_tif_file_path):                            # verify that the file exist
    logging.error('Calcium video cropped file not found.')

In [ ]:
# videos visualization
FOV = cm.load(input_tif_file_path_FOV)           # load complete FOV

gain = 1.         #light intensity of video
magnification = 1 #size of display movie
fr = 20           #frequency of visualization (it is not sampling rate from the original movie)

FOV.play(gain = gain, magnification = magnification, fr = fr) #show video of calcium fluorecence

In [ ]:
# image visualization
cropped_region_fig_path = figures_dir + 'FOV_calcium_video_1.png'       #path where the image will be saved

auxiliary_functions.plot_FOV(FOV_file = input_tif_file_path_FOV , ROI_file = input_tif_file_path, 
                             output_file = cropped_region_fig_path)

For any process that tries to separate neurons from background it is important to take into account that the statistics of regions with neurons will be remarkably different from the statistics of regions with no neurons. With  1-photon imaging this is diffuse because background also includes activity from neurons in other focal planes (as explained above). 

### Exercise 0. Image characteristics

> Fluctuations in the activitation of the neuron can be directly observed on the video or in a temporal trace plot. Plotting the temporal trace will allow us to see the calcium transient dinamic, if a pixel belonging to a neuron is selected. Select a set of *N* random pixels from the ROI and plot the temporal evolution of pixel value. If the random pixels are choosen from an active neuron, you will be able to see the calcium transcient dynamic. 

***Answer:***

In [ ]:
temporal_pixel_evolution_fig_path = figures_dir + 'temporal_evolution_calcium_video_1.png'

auxiliary_functions.temporal_evolution(file_name=input_tif_file_path, output_file_name=temporal_pixel_evolution_fig_path)

## 1 Videos

### Exercise 1. Video statistics

> Compute the mean and correlation summary images for *caiman_video_trial_0.tif*. You can use *mean* and *corrcoef* functions from *numpy* for that aim. Note: Correlation image can be computed taking *m* nearby pixels. At this instance use 4 nearest-neighbords for each pixel. When using CaImAn we will use 8 nearest-neightbords. Plot the resulting summary images and the difference between them (make sure they are defined in the same range of relative values), and plot all in the same range [0,1].

***Answer:***

In [ ]:
''' Construct functions '''

def summary_image_mean(video):
    mean_im = np.mean(video, axis=0)
    return mean_im
    
def summary_image_correlation(video):
    video = video.T
    cs = np.zeros((video.shape[0:2]))
    for i in range(1, video.shape[0]-1):
        for j in range(1, video.shape[1]-1):
            file = video[i-1:i+2, j-1:j+2, :]
            file = file.reshape(9, video.shape[2])
            temp = np.corrcoef(file)
            cs[i,j] = np.mean(temp[1:,0])
    return cs.T

In [ ]:
''' Compute mean and correlation of summary images '''
original_movie = cm.load(input_tif_file_path)
mean_frame = summary_image_mean(original_movie)
correlation_image = summary_image_correlation(original_movie)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axs[0].set_title('Mean image')
axs[0].imshow(mean_image, cmap='gray')
axs[1].set_title('Correlation image')
axs[1].imshow(corr_image, cmap='gray')

In [ ]:
## plotting code is provided for once the mean and correlation summary images are computed
figure, axes = plt.subplots(1,3)

axes[0].set_xlabel('x', fontsize = 12)
axes[0].set_ylabel('y', fontsize = 12)
axes[0].set_title('Mean Summary Image', fontsize = 15)

axes[1].set_xlabel('x', fontsize = 12)
axes[1].set_ylabel('y', fontsize = 12)
axes[1].set_title('Correlation Summary Image', fontsize = 15)

axes[1].set_xlabel('x', fontsize = 12)
axes[1].set_ylabel('y', fontsize = 12)
axes[2].set_title('MSI - CSI', fontsize = 15)

mesh0 = axes[0].pcolormesh(mean_frame/np.max(mean_frame), cmap = 'viridis')
mesh1 = axes[1].pcolormesh(correlation_image, cmap = 'viridis')
mesh2 = axes[2].pcolormesh(mean_frame/np.max(mean_frame)-correlation_image, cmap = 'viridis')

vmin_corr = 0
vmax_corr = 1
mesh0.set_clim(vmin_corr,vmax_corr)
mesh1.set_clim(vmin_corr,vmax_corr)
mesh2.set_clim(vmin_corr,vmax_corr)


figure.colorbar(mesh0,ax=axes[0])
figure.colorbar(mesh1,ax=axes[1])
figure.colorbar(mesh2,ax=axes[2])

figure.set_size_inches(15,5)

## 2 Preprocessing calcium videos

In [ ]:
## parameters_motion_correction. We create them in a dictionary to prepare for caiman paramenters requirements

parameters_motion_correction = { 'pw_rigid': True,  # flag for performing piecewise-rigid motion correction (otherwise just rigid)
                                'save_movie_rig': True, # flag to say whether the rigig motion corrected movie is saved
                                'gSig_filt': (5, 5),     # size of high pass spatial filtering, used in 1p data
                                'max_shifts': (25, 25),  # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
                                'niter_rig': 1 ,
                                'strides': (48, 48),     # create a new patch every x pixels for pw-rigid correction
                                'overlaps': (96, 96),    # overlap between pathes (size of patch strides+overlaps)
                                'upsample_factor_grid': 1,
                                'num_frames_split': 80,  # length in frames of each chunk of the movie (to be processed in parallel)
                                'max_deviation_rigid': 15, # maximum deviation allowed for patch with respect to rigid shifts
                                'shifts_opencv': True, 'use_cuda': False, 'nonneg_movie': True,
                                'border_nan': 'copy'}

# load tif movie as a caiman movie object
original_movie = cm.load(input_tif_file_path)
# Calculate movie minimum to subtract from movie (this is necesary for running motion correction)
min_mov = np.min(original_movie)
# Apply the parameters to the CaImAn algorithm
parameters_motion_correction['min_mov'] = min_mov
opts = params.CNMFParams(params_dict = parameters_motion_correction)

In [ ]:
n_processes = psutil.cpu_count()
cm.cluster.stop_server()
# Start a new cluster
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=n_processes, single_thread=False)

In [ ]:
#create an object for motion correction
mc = MotionCorrect(input_tif_file_path, dview = dview, **opts.get_group('motion'))
# Perform rigid motion correction
mc.motion_correct_rigid(save_movie = parameters_motion_correction['save_movie_rig'], template = None)
m_rig = cm.load(mc.fname_tot_rig[0])
fname_tot_rig = m_rig.save('calcium_video_1' +'_rig' + '.mmap',  order='C')
# os.remove(os.path.join(mc.fname_tot_rig[0]))

#perform non-rigid motion correction
total_template_rig = mc.total_template_rig
mc.motion_correct_pwrigid(save_movie=True, template = total_template_rig)
fname_tot_els = mc.fname_tot_els[0]
m_els = cm.load(fname_tot_els)
fname_tot_els  = m_els.save('calcium_video_1' + '_els' + '.mmap',  order='C')
# os.remove(mc.fname_tot_els[0])

### Exercise 2. Rigid vs Non-Rigid motion correction

> The objects *mc* contains information regarding the shifts produced by rigid and non-rigid motion corrections. For rigid motion correction you will find two vectors of size *500* frames corresponding to the shifts produced in each frame by rigid displacements. For non-rigid motion correction each atribute of the object contains multiple size *500* vectors corresponding to the shifts of all patches in *x* and *y* direction. 
 * Plot the shifts produced by rigid and pw-rigid motion correction frame by frame. In the case of pw-rigid plot the mean and std from multiple patches in each direction.
 * Compare rigid and no-rigid shifts statistics. Plot histogram for both and compute relevants parametes. 
 * Which mc algorithm would you expect to produce better outputs respect to the quality of the motion corrected image and why?

Note: *use mc.shifts_rig, mc.x_shifts_els and mc.y_shifts_els*

***Answer:***

- I would expect the pw-rigid motion correction to produce better outputs because it can shift various parts of the image over distinct distances. Thus, it is more flexible (has more degrees of freedom).
- From the plots below, the difference in outputs between the methods is quite small. 

In [ ]:
''' Get the motion correction shifts '''
rig = np.array(mc.shifts_rig)
els_x = np.array(mc.x_shifts_els)
els_y = np.array(mc.y_shifts_els)

In [ ]:
''' Use interactive plot to visualize the histograms '''
def vis_hists(i, num_bins = 20):
    fig, axs = plt.subplots(1, 2, figsize=(15,5))
    
    axs[0].set_title('Shifts in the x-direction')
    axs[0].hist(els_x[i,:],bins=num_bins)
    axs[0].axvline(rig[i,0], label='Rigid correction')
    axs[0].axvline(np.mean(els_x[i,:]), label='Mean pw-rigid correction')
    axs[0].legend()
    
    axs[1].set_title('Shifts in the y-direction')
    axs[1].hist(els_y[i,:],bins=num_bins)
    axs[1].axvline(rig[i,1], label='Rigid correction')
    axs[1].axvline(np.mean(els_y[i,:]), label='Mean pw-rigid correction')
    axs[1].legend()
    
    fig.suptitle(f'Frame number {i}')
    plt.show()
interact(vis_hists, i=widgets.IntSlider(min=0, max=(rig.shape[0]), value=0))

### Exercise 3. Quality assesment

> Quality assesment by crispness is computed in a summary image that could be any of the ones previously mentioned.  
* Compute the mean and correlation image for the original movie, the rigid motion corrected and the pw-motion corrected. You can use your previously done function or use the method local correlations from caiman object. You can see documantion and caiman demostration for this. 
* Plot the different summary images for original movie, rigid-motion corrected movie and pw-rigid motion corrected movie.
* Create a function that computes the crispness as define above and use it to compute quality measure in the mean and correlation summary images. For that, just remove the border in the computations. Compare the crispness result after motion correction with the original value for the correlation image.
* Defined as above, can cripsness be used directly to compare the motion quality of differnt videos with different image resolutions? what should be taken into account in order to compare them?

***Answer:***

On first inspection, the images are not that different, though the motion corrected images do seem more blurry. When we compute the crispness it is clear that the motion corrected images have lower crispness values, indicating a reduction in quality.

In [ ]:
''' Compute summary images '''
original_movie_mean = summary_image_mean(original_movie)
m_rig_mean = summary_image_mean(m_rig)
m_els_mean = summary_image_mean(m_els)

''' Complete code correlation summary image using caiman with 8 neighbours'''
original_movie_corr = summary_image_corr(original_movie)
m_rig_corr = summary_image_corr(m_rig)
m_els_corr = summary_image_corr(m_els)

Quality is measured by crispness, this is implemented in the  Caiman software.

In [ ]:
''' Compute crispness '''
bord_px_rig = np.ceil(np.max(mc.shifts_rig)).astype(np.int)
bord_px_els = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
final_size = np.subtract(mc.total_template_els.shape, 2 * bord_px_els) 

tmpl_rig, correlations_orig, flows_orig, norms_orig, crispness_orig = cm.motion_correction.compute_metrics_motion_correction(input_tif_file_path, final_size[0], final_size[1], swap_dim=False, winsize=100, play_flow=False)
tmpl_rig, correlations_rig, flows_rig, norms_rig, crispness_rig = cm.motion_correction.compute_metrics_motion_correction(fname_tot_rig, final_size[0], final_size[1], swap_dim=False, winsize=100, play_flow=False)
tmpl_els, correlations_els, flows_els, norms_els, crispness_els = cm.motion_correction.compute_metrics_motion_correction(fname_tot_els, final_size[0], final_size[1], swap_dim=False, winsize=100, play_flow=False)

print(f'Original crispness {crispness_orig}')
print(f'Rigid crispness {crispness_rig}')
print(f'Elastic crispness {crispness_elastic}')

In [ ]:
figure, axes = plt.subplots(2,3)
figure.set_size_inches(15,7)

mesh0 = axes[0,0].pcolormesh(original_movie_mean/np.max(original_movie_mean))
axes[0,0].set_title('Original movie mean')
mesh1 = axes[0,1].pcolormesh(m_rig_mean/np.max(m_rig_mean))
axes[0,1].set_title('Rigig MC mean')
mesh2 = axes[0,2].pcolormesh(m_els_mean/np.max(m_els_mean))
axes[0,2].set_title('PW-rigid MC mean')

mesh3 = axes[1,0].pcolormesh(original_movie_corr)
axes[1,0].set_title('Original movie correlation')
mesh4 = axes[1,1].pcolormesh(m_rig_corr)
axes[1,1].set_title('Rigid MC correlation')
mesh5= axes[1,2].pcolormesh(m_els_corr)
axes[1,2].set_title('PW-Rigid MC correlation')

vmin_corr = 0
vmax_corr = 1
mesh0.set_clim(vmin_corr,vmax_corr)
mesh1.set_clim(vmin_corr,vmax_corr)
mesh2.set_clim(vmin_corr,vmax_corr)
mesh3.set_clim(vmin_corr,vmax_corr)
mesh4.set_clim(vmin_corr,vmax_corr)
mesh5.set_clim(vmin_corr,vmax_corr)

figure.colorbar(mesh0,ax=axes[0,0])
figure.colorbar(mesh1,ax=axes[0,1])
figure.colorbar(mesh2,ax=axes[0,2])
figure.colorbar(mesh3,ax=axes[1,0])
figure.colorbar(mesh4,ax=axes[1,1])
figure.colorbar(mesh5,ax=axes[1,2])

#### Example of source extraction

Here we will run an example on the video we have been working on for source extraction. First we will define some fixed parameters for this example and later we will explore the impact of those in the source extraction output (particularly the seeds for inicialization in CNMF-E). 

In [ ]:
# run source extraction (fit the model to the data)
cnmf_object = cnmf.CNMF(n_processes=n_processes, dview=dview, params=opts)
cnmf_object.fit(images)

In [ ]:
# Save the cnmf object as a hdf5 file
output_cnmf_file_path = data_dir + 'source_extracted/' + 'calcium_video_0_cnmf.hdf5'
cnmf_object.save(output_cnmf_file_path)

The main outputs from CNMF are the matrix **A** containing the footprints and the matrix **C** containing the temporal calcium traces. Here we show the result of the A matrix by ploting the footprints on top on the correlation image and also ploting the temporal traces. 

*Note: Footprints are save in cnmf_object.estimates.A and temporal traces in cnmf_object.estimates.C *

In [ ]:
# display footprints in the ROI
figure, axes = plt.subplots(1)
pos = axes.imshow(corr_image)
coordinates = cm.utils.visualization.get_contours(cnmf_object.estimates.A, np.shape(corr_image), 0.2, 'max')
for c in coordinates:
    v = c['coordinates']
    c['bbox'] = [np.floor(np.nanmin(v[:, 1])), np.ceil(np.nanmax(v[:, 1])),
                    np.floor(np.nanmin(v[:, 0])), np.ceil(np.nanmax(v[:, 0]))]
    axes.plot(*v.T, c='r')
figure.colorbar(pos, ax=axes)

print('Total number of componentes = ' + f'{cnmf_object.estimates.A.shape[1]}')

In [ ]:
# plot temporal traces
figure, axes = plt.subplots(1)
C_0 = cnmf_object.estimates.C.copy()
C_0[1] += C_0[0].min()
for i in range(1, len(C_0)):
    C_0[i] += C_0[i].min() + C_0[:i].max()
    axes.plot(C_0[i])
axes.set_xlabel('t [frames]')
axes.set_yticks([])
axes.set_ylabel('activity')
figure.set_size_inches([50., .5 * len(C_0)])

### Exercise 5. Source extraction parameters impact

> * For a fixed minimum correlation value (ie 0.6) select a range of possible PNR minimun values and study the impact of those in the final source extraction output.
  * For a fixed value of PNR (ie 6) select a range of minimum correlation value and study the impact of those in the final source extraction output.


> * How do these parameters affect the spatial sparcity of the output? Which set of parameter selection procedes the higher number of neurons? Is maximal number of neurons always better? Why?  

Smaller values for both the minimal value of the peak correlation images and peak noise ratio result in the detection of more neurons. If the value is too low the activity overlaps, which is a bad sign and may indicate noise. A maximal number of neurons is not always better because they may be false positives. We should therefore carefully select the values.

> * Generate a plot of number of neurons vs PNR (or min corr value) to show how these parameters affect the source extracted signals.


> * Are the temporal traces affected by these parameters?

Yes.


***Answer:***

We first iterate over the correlation values 

In [ ]:
corr_values = np.linspace(0, 1, 6)

In [ ]:
parameters_source_extraction ={ 'fr': 10,               # movie frame rate
                               'decay_time': 0.1,       # length of a typical transient in seconds
                               'min_corr': 0.7,         # min peak value from correlation image
                               'min_pnr': 7,            # min peak to noise ration from PNR image
                                'p': 1,                 # order of the autoregressive system 
                               'K': None,               # upper bound on number of components per patch, in general None
                               'gSig': (4, 4),          # gaussian width of a 2D gaussian kernel, which approximates a neuron
                               'gSiz': (17, 17),        # average diameter of a neuron, in general 4*gSig+1
                               'ring_size_factor': 1.4, # radius of ring is gSiz*ring_size_factor
                               'merge_thr': 0.7, 'rf': 60,
                               'stride': 30, 'tsub': 1, 'ssub': 2, 'p_tsub': 1, 'p_ssub': 2, 'low_rank_background': None,
                               'nb': 0, 'nb_patch': 0, 'ssub_B': 2, 'init_iter': 2,
                               'method_init': 'corr_pnr', 'method_deconvolution': 'oasis',
                               'update_background_components': True,
                               'center_psf': True, 'border_pix': 0, 'normalize_init': False,
                               'del_duplicates': True, 'only_init': True}

fig, axes = plt.subplots(2, 3, figsize=(20, 15))

for ax, min_corr in zip(axes.flatten(), corr_values):
    parameters_source_extraction['min_corr'] = min_corr
    opts = params.CNMFParams(params_dict=parameters_source_extraction)
    cnmf_object = cnmf.CNMF(n_processes=n_processes, dview=dview, params=opts)
    cnmf_object.fit(images)
    pos = ax.imshow(corr_image)
    coordinates = cm.utils.visualization.get_contours(cnmf_object.estimates.A, np.shape(corr_image), 0.2, 'max')
    for co in coordinates:
        v = co['coordinates']
        co['bbox'] = [np.floor(np.nanmin(v[:, 1])), np.ceil(np.nanmax(v[:, 1])), np.floor(np.nanmin(v[:, 0])), np.ceil(np.nanmax(v[:, 0]))]
        ax.plot(*v.T, c='r')
    ax.set_title(f'Min correlation footprints {min_corr} | number of cells {cnmf_object.estimates.A.shape[1]}')

Then iterate over the PNR values

In [ ]:
corr_values = np.linspace(0, 20, 6)

In [ ]:
parameters_source_extraction ={ 'fr': 10,               # movie frame rate
                               'decay_time': 0.1,       # length of a typical transient in seconds
                               'min_corr': 0.7,         # min peak value from correlation image
                               'min_pnr': 7,            # min peak to noise ration from PNR image
                                'p': 1,                 # order of the autoregressive system 
                               'K': None,               # upper bound on number of components per patch, in general None
                               'gSig': (4, 4),          # gaussian width of a 2D gaussian kernel, which approximates a neuron
                               'gSiz': (17, 17),        # average diameter of a neuron, in general 4*gSig+1
                               'ring_size_factor': 1.4, # radius of ring is gSiz*ring_size_factor
                               'merge_thr': 0.7, 'rf': 60,
                               'stride': 30, 'tsub': 1, 'ssub': 2, 'p_tsub': 1, 'p_ssub': 2, 'low_rank_background': None,
                               'nb': 0, 'nb_patch': 0, 'ssub_B': 2, 'init_iter': 2,
                               'method_init': 'corr_pnr', 'method_deconvolution': 'oasis',
                               'update_background_components': True,
                               'center_psf': True, 'border_pix': 0, 'normalize_init': False,
                               'del_duplicates': True, 'only_init': True}

fig, axes = plt.subplots(2, 3, figsize=(20, 15))

for ax, min_pnr in tqdm(zip(axes.flatten(),corr_values)):
    parameters_source_extraction['min_pnr'] = min_pnr
    opts = params.CNMFParams(params_dict=parameters_source_extraction)
    cnmf_object = cnmf.CNMF(n_processes=n_processes, dview=dview, params=opts)
    cnmf_object.fit(images)
    pos = ax.imshow(corr_image)
    coordinates = cm.utils.visualization.get_contours(cnmf_object.estimates.A, np.shape(corr_image), 0.2, 'max')
    for co in coordinates:
        v = co['coordinates']
        co['bbox'] = [np.floor(np.nanmin(v[:, 1])), np.ceil(np.nanmax(v[:, 1])), np.floor(np.nanmin(v[:, 0])), np.ceil(np.nanmax(v[:, 0]))]
        ax.plot(*v.T, c='r')
    ax.set_title(f'Min pnr footprints {min_pnr} | number of cells {cnmf_object.estimates.A.shape[1]}')

### Exercise 6. Photobleaching effect

> * Study the effect of photobleaching in the raw signal. For that aim use the videos called *'caiman_video_trial_'+ f'{trial}' +'.tif'* to create a new multisession video.
>* Repeat exercise 0 in this new signal. How is bleaching reflected in the temporal evolution of a set of random pixels for these signals? 


***Answer:***

- In the figure below we can see that there is indeed an effect of the prolonged light exposure on the darkness of the images. The pixel values become darker as time progresses, shown in the temporal evolution and also in the histograms (skewed distributions to the left).
- The peaks in the temporal evolutions likely indicate the start of a new trial.
- So, photobleaching does affects the calcium imaging temporal traces, which introduces a challenge in analyzing the data. The bleaching is reflected in the downwards trend in the pixel valaues. This should be taken into account. 


In [ ]:
# create a movie concatenating all segments on multiple trials
output_tif_file_concatenated = data_dir + 'raw/' + 'movie_concatenated.tif'
movie_list = [data_dir + 'raw/' + 'caiman_video_trial_' + str(i) + '.tif' for i in range(4+1)]

# complete with concatenation
concatenated_movie = load_movie_chain(movie_list)
concatenated_file_path = data_dir + 'raw/' + 'concatenated_video.tif'
concatenated_movie.save(concatenated_file_path)

In [ ]:
# temporal evolution visualization and statistics of pixels
temporal_pixel_evo_fig_path = figures_dir + 'temporal_evolution_calcium_video_concatenated.png'

auxiliary_functions.temporal_evolution(file_name=concatenated_file_path, output_file_name=temporal_pixel_evo_fig_path)